In [6]:
from IPython.display import display
import os
import logging
import re
import subprocess
from google.colab import files

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Install and import pdfplumber
try:
    import pdfplumber
except ImportError:
    subprocess.check_call(["pip", "install", "pdfplumber"])
    import pdfplumber

# Load spaCy and set max_length after loading
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
    nlp.max_length = 3_000_000
except ImportError:
    logging.warning("Downloading en_core_web_sm model for spaCy...")
    subprocess.check_call(["python3", "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")
    nlp.max_length = 3_000_000
except Exception as e:
    logging.error(f"Error loading spaCy model: {e}")
    raise

# Load document text
def load_document(filepath):
    try:
        if filepath.endswith('.txt'):
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
                return file.read()
        elif filepath.endswith('.pdf'):
            with pdfplumber.open(filepath) as pdf:
                return "\n".join(
                    page.extract_text() for page in pdf.pages if page.extract_text()
                )
        else:
            logging.warning(f"Unsupported file type: {filepath}")
            return ""
    except Exception as e:
        logging.error(f"Error reading file {filepath}: {e}")
        return ""

# Extract sentences with user-provided keywords
def find_requirements_in_text(text, keywords):
    if not text:
        return []
    requirements = []
    for i, sent in enumerate(nlp(text).sents):
        for keyword in keywords:
            if re.search(keyword, sent.text, re.IGNORECASE):
                requirements.append((sent.text.strip(), i + 1))
                break
    return requirements

# Main analysis flow
def main(folder_path="my_colab_folder"):
    os.makedirs(folder_path, exist_ok=True)

    # Check if any supported files already exist
    existing_files = [
        f for f in os.listdir(folder_path)
        if f.endswith(('.txt', '.pdf'))
    ]

    if existing_files:
        logging.info("Files already found. Skipping upload.")
    else:
        uploaded = files.upload()
        for filename, content in uploaded.items():
            filepath = os.path.join(folder_path, filename)
            with open(filepath, 'wb') as f:
                f.write(content)
            logging.info(f"Saved uploaded file: {filepath}")
        existing_files = [
            f for f in os.listdir(folder_path)
            if f.endswith(('.txt', '.pdf'))
        ]

    # Get keywords from user input
    keywords_input = input("Enter keywords separated by commas: ")
    keywords = [k.strip() for k in keywords_input.split(',') if k.strip()]

    # Process files
    for filename in existing_files:
        filepath = os.path.join(folder_path, filename)
        logging.info(f"Processing: {filepath}")
        text = load_document(filepath)
        found_requirements = find_requirements_in_text(text, keywords)

        print(f"\n== Results from: {filename} ==")
        if found_requirements:
            for req, sent_num in found_requirements:
                print(f"- Sentence {sent_num}: {req}")
        else:
            print("No matching requirements found.")

# Run main
if __name__ == "__main__":
    main()


Enter keywords separated by commas: echo



== Results from: PCM80_QRG_Rev1.pdf ==
- Sentence 27: Use SELECT sive set of informative dis-
effects) and any time-based echoes/beats which are and Up and Down to locate play messages which can be
modulator can be individu- linked to tempo.
- Sentence 40: P0 P1 P2 P3
Matrix Matrix Matrix Matrix
Location Preset Type Location Preset Type Location Preset Type Location Preset Type
P0
0.0-1.9 Multi Effects 0.0-3.6 Rhythmic Echo and 0.0-1.0 EQ Effects 0.0-2.8 Reverb Effects
2.0-3.5 Modulation Effects Delay Effects 1.1-2..4 Spatial Effects 2.9-3.9 Processed Reverb
3.6-4.9 Special Effects 3.7-4.9 Ambience Effects 2.5-4.0 Gain Effects Effects
4.1-4.9 Resonant Chord 4.0-4.9 Remix Effects
Effects
Compare
Each preset has one or more Turn ADJUST to display the Continue turning ADJUST to
parameters patched to the name and the current value change the value of the patch
ADJUST knob.



== Results from: PCM80_DualFX_Preset_Menu (1).pdf ==
- Sentence 14: Hi Cut 0–127 3.3 15ips Echo ADJUST: Feedback 0–100
Program Bank X0
A stereo chamber optimized for drum submix, followed by a Stereo 15ips tape echo simulation.
- Sentence 18: Lo Cut 0–127 3.4 7.5ipsEkoRvb ADJUST: Feedback 0–100
The complement of Mix>Drum>LP with the drum chamber Stereo plate reverb fed by a 7.5ips tape echo simulation.
- Sentence 31: A variation where the tape echoes also serve as predelay for
0.1 Left Room ADJUST:
- Sentence 43: Eko Spills 1 ADJUST: Depth 1–100
0.3 Organ Room ADJUST: Slow/Fast 0–1
Echoes that drop in pitch as the input signal fades.
- Sentence 56: Amb1 is a triplet (100) echoes.
- Sentence 61: Amb3 is a tight space with prominent chamber is in parallel with the echo effect.
- Sentence 65: A variation with the rhythm changing from quarter-notes to
A light touch of triplet echoes is added to the reverb in front of
sixteenths.
- Sentence 67: Echoes and tremolo are tempo-
2.1 Mix>Amb1>LP


== Results from: PCM80_DualFX_Preset_Menu.pdf ==
- Sentence 14: Hi Cut 0–127 3.3 15ips Echo ADJUST: Feedback 0–100
Program Bank X0
A stereo chamber optimized for drum submix, followed by a Stereo 15ips tape echo simulation.
- Sentence 18: Lo Cut 0–127 3.4 7.5ipsEkoRvb ADJUST: Feedback 0–100
The complement of Mix>Drum>LP with the drum chamber Stereo plate reverb fed by a 7.5ips tape echo simulation.
- Sentence 31: A variation where the tape echoes also serve as predelay for
0.1 Left Room ADJUST:
- Sentence 43: Eko Spills 1 ADJUST: Depth 1–100
0.3 Organ Room ADJUST: Slow/Fast 0–1
Echoes that drop in pitch as the input signal fades.
- Sentence 56: Amb1 is a triplet (100) echoes.
- Sentence 61: Amb3 is a tight space with prominent chamber is in parallel with the echo effect.
- Sentence 65: A variation with the rhythm changing from quarter-notes to
A light touch of triplet echoes is added to the reverb in front of
sixteenths.
- Sentence 67: Echoes and tremolo are tempo-
2.1 Mix>Amb1>LP ADJ


== Results from: PCM80_Presets_Rev1 (1).pdf ==
- Sentence 15: ImageKnob 0–12
Six rhythmic echo voices are EQ’d and panned across stereo Individual input notes come out at random locations in
Prime Blue ADJUST: Efx/Rvb X 0–127 space.
- Sentence 18: Delays "creep" slowly out to new time valu
ee stereo effects: 6-voice chorus, rhythmic echoes, and Best with solo instruments or voices.
- Sentence 20: ADJUST changes the effect smoothly from chorus, to 1.9 ADJUpMyEchos ADJUST:
- Sentence 21: EchosLvl 0–10
rus with echoes, to chorus with echoes and reverb, to Concert hall reverb with 4 panned echo voices in the back-
erb with echoes, then to reverb only.
- Sentence 24: DrownKnob 0–10
-voice stereo chorus with echoes and stereo reverb.
- Sentence 27: Wet Chorus ADJUST: FX Mix 0–100 2.0 FSw2 Elevate ADJUST: Feedback 0–99 3.7 Thunder FX ADJUST: FlashTime 0–12
ick wet chorus combined with a small amount of reverb to Multiple echo voices with diffusion, modulation and a touch of Produces a rollin


== Results from: PCM80_Presets_Rev1.pdf ==
- Sentence 15: ImageKnob 0–12
Six rhythmic echo voices are EQ’d and panned across stereo Individual input notes come out at random locations in
Prime Blue ADJUST: Efx/Rvb X 0–127 space.
- Sentence 18: Delays "creep" slowly out to new time valu
ee stereo effects: 6-voice chorus, rhythmic echoes, and Best with solo instruments or voices.
- Sentence 20: ADJUST changes the effect smoothly from chorus, to 1.9 ADJUpMyEchos ADJUST:
- Sentence 21: EchosLvl 0–10
rus with echoes, to chorus with echoes and reverb, to Concert hall reverb with 4 panned echo voices in the back-
erb with echoes, then to reverb only.
- Sentence 24: DrownKnob 0–10
-voice stereo chorus with echoes and stereo reverb.
- Sentence 27: Wet Chorus ADJUST: FX Mix 0–100 2.0 FSw2 Elevate ADJUST: Feedback 0–99 3.7 Thunder FX ADJUST: FlashTime 0–12
ick wet chorus combined with a small amount of reverb to Multiple echo voices with diffusion, modulation and a touch of Produces a rolling cl